In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from collections import defaultdict, Counter
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import tree, linear_model, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, fbeta_score, roc_auc_score
from sklearn.model_selection import cross_val_score
#from sklearn.ensemble import AdaBoostClassifier

from sklearn.datasets import make_classification


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler

#Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn import grid_search
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.cross_validation import train_test_split, ShuffleSplit


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Read the dataset
df1_all = pd.read_csv('takehome_users.csv', encoding = "ISO-8859-1")
df2_all = pd.read_csv('takehome_user_engagement.csv', parse_dates=['time_stamp'])

In [3]:
def Print_Stats(df, num):
    #Prints basic summary statistics of a pandas dataframe    

    print(df.head(num))
    print(df.shape)
    print(df.info())
    print(df.columns)
    print(df.describe())
    
def Print_Frequency(df, bins = []):
    #Prints frequency distribution
    
    col_list = list(df.columns.values)
    
    if bins == []:
        for col in col_list:
            print(df[col].value_counts().sort_index())
    else:
        for col in col_list:
            print(pd.cut(df[col],bins).value_counts().sort_index())

In [4]:
Print_Stats(df1_all, 10)

   object_id        creation_time               name  \
0          1  2014-04-22 03:53:30     Clausen August   
1          2  2013-11-15 03:45:04      Poole Matthew   
2          3  2013-03-19 23:14:52  Bottrill Mitchell   
3          4  2013-05-21 08:09:28    Clausen Nicklas   
4          5  2013-01-17 10:14:20          Raw Grace   
5          6  2013-12-17 03:37:06      Cunha Eduardo   
6          7  2012-12-16 13:24:32       Sewell Tyler   
7          8  2013-07-31 05:34:02  Hamilton Danielle   
8          9  2013-11-05 04:04:24         Amsel Paul   
9         10  2013-01-16 22:08:03       Santos Carla   

                           email    creation_source  \
0       AugustCClausen@yahoo.com       GUEST_INVITE   
1         MatthewPoole@gustr.com         ORG_INVITE   
2     MitchellBottrill@gustr.com         ORG_INVITE   
3      NicklasSClausen@yahoo.com       GUEST_INVITE   
4             GraceRaw@yahoo.com       GUEST_INVITE   
5  EduardoPereiraCunha@yahoo.com       GUEST_INVITE  

In [5]:
myDict = {}
for _, row in df2_all.iterrows():
  if row['user_id'] not in myDict.keys():
      myDict[row['user_id']] = 0
      current_id = row['user_id']
      count = 1
      currant_date = "2013-04-22 03:53:30"
    
  elif (row['user_id'] == current_id and count <3):
    
      if currant_date == "2013-04-22 03:53:30":
        currant_date = row['time_stamp']
      elif ((row['time_stamp'] - currant_date).days <= 7 and (row['time_stamp'] - currant_date).days > 0):
        count += 1
        if count == 2:
          next_date = row['time_stamp']
        elif count == 3:
          myDict[row['user_id']] = 1
      elif ((row['time_stamp'] - currant_date).days > 7):
        if count == 1:
          currant_date = row['time_stamp']
          count = 1
        elif count == 2:
          currant_date = next_date
          if ((row['time_stamp'] - currant_date).days <= 7 and (row['time_stamp'] - currant_date).days > 0):
            count = 2
          elif ((row['time_stamp'] - currant_date).days > 7):
            currant_date = row['time_stamp']
            count == 1
        
       
    
       

In [2]:
#for keys, value in myDict.items():
    #print(keys, value)

In [7]:
df_target = pd.DataFrame(list(myDict.items()), columns=['object_id', 'target'])
Print_Stats(df_target, 10)

   object_id  target
0          1       0
1          2       1
2          3       0
3          4       0
4          5       0
5          6       0
6          7       0
7         10       1
8         11       0
9         13       0
(8823, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8823 entries, 0 to 8822
Data columns (total 2 columns):
object_id    8823 non-null int64
target       8823 non-null int64
dtypes: int64(2)
memory usage: 137.9 KB
None
Index(['object_id', 'target'], dtype='object')
          object_id       target
count   8823.000000  8823.000000
mean    6019.821716     0.194265
std     3464.251001     0.395656
min        1.000000     0.000000
25%     3017.500000     0.000000
50%     6034.000000     0.000000
75%     9029.500000     0.000000
max    12000.000000     1.000000


In [8]:
# Merge datasets

df_merge = pd.merge(df1_all, df_target, on='object_id', how='left')

#print(df_merge[['object_id', 'Flag']])

df_merge['target'].fillna(0, inplace=True)
#print(df_merge[['object_id', 'Flag']])

#Print_Stats(df_merge, 5)

Print_Frequency(df_merge[['creation_source']])



GUEST_INVITE          2163
ORG_INVITE            4254
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64


In [9]:
Print_Stats(df_merge, 10)

   object_id        creation_time               name  \
0          1  2014-04-22 03:53:30     Clausen August   
1          2  2013-11-15 03:45:04      Poole Matthew   
2          3  2013-03-19 23:14:52  Bottrill Mitchell   
3          4  2013-05-21 08:09:28    Clausen Nicklas   
4          5  2013-01-17 10:14:20          Raw Grace   
5          6  2013-12-17 03:37:06      Cunha Eduardo   
6          7  2012-12-16 13:24:32       Sewell Tyler   
7          8  2013-07-31 05:34:02  Hamilton Danielle   
8          9  2013-11-05 04:04:24         Amsel Paul   
9         10  2013-01-16 22:08:03       Santos Carla   

                           email    creation_source  \
0       AugustCClausen@yahoo.com       GUEST_INVITE   
1         MatthewPoole@gustr.com         ORG_INVITE   
2     MitchellBottrill@gustr.com         ORG_INVITE   
3      NicklasSClausen@yahoo.com       GUEST_INVITE   
4             GraceRaw@yahoo.com       GUEST_INVITE   
5  EduardoPereiraCunha@yahoo.com       GUEST_INVITE  

In [10]:
# Define one-hot encoder function
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [16]:
# Create dummy variables for categorical variables 'creation_source' using the function one_hot
df = one_hot(df_merge, ['creation_source'])

Print_Stats(df, 10)

df_freq = df[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip' , 'target']]

Print_Frequency(df_freq)

   object_id        creation_time               name  \
0          1  2014-04-22 03:53:30     Clausen August   
1          2  2013-11-15 03:45:04      Poole Matthew   
2          3  2013-03-19 23:14:52  Bottrill Mitchell   
3          4  2013-05-21 08:09:28    Clausen Nicklas   
4          5  2013-01-17 10:14:20          Raw Grace   
5          6  2013-12-17 03:37:06      Cunha Eduardo   
6          7  2012-12-16 13:24:32       Sewell Tyler   
7          8  2013-07-31 05:34:02  Hamilton Danielle   
8          9  2013-11-05 04:04:24         Amsel Paul   
9         10  2013-01-16 22:08:03       Santos Carla   

                           email    creation_source  \
0       AugustCClausen@yahoo.com       GUEST_INVITE   
1         MatthewPoole@gustr.com         ORG_INVITE   
2     MitchellBottrill@gustr.com         ORG_INVITE   
3      NicklasSClausen@yahoo.com       GUEST_INVITE   
4             GraceRaw@yahoo.com       GUEST_INVITE   
5  EduardoPereiraCunha@yahoo.com       GUEST_INVITE  

Modelling

In [19]:
# Creating feature and target data set
#df = df.rename(columns={"city_King's Landing" : 'city_KingsLanding'})
#df_modeling = df.drop(['city', 'phone', 'phone_iPhone', 'city_KingsLanding', 'surge_pct'])


df_modeling = df[['creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE', 'creation_source_PERSONAL_PROJECTS',
                 'creation_source_SIGNUP' ,'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'target']]

# Feature matrix
feature = df_modeling.drop(['target'], axis=1)

# Target variable
target = df_modeling['target']

In [17]:
# Define a function to fit and print results

def Model(X_train, y_train, X_test, y_test, classifier):
    reg = classifier.fit(X_train, y_train)
    
    # Compute metrics
    y_pred = reg.predict(X_test)
    score = r2_score(y_test, y_pred)
    score_1 = reg.score(X_test, y_test)
    accuracy_test = accuracy_score(y_test, y_pred)
    roc_score = roc_auc_score(y_test, y_pred)
    accuracy_train = accuracy_score(y_train, reg.predict(X_train))
    
    print('Accuracy score on training data: {:.4f}'.format(accuracy_train))
    print('Accuracy score on testing data: {:.4f}'.format(accuracy_test))
    print('')    
    
    
    print('Roc score: {:.4f}'.format(roc_score))
    print('')

    # Generate the confusion matrix and classification report
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))
    print('')
    
    print('Classification Report')
    print(classification_report(y_test, y_pred))

    return reg

In [20]:
# Split the data set into train and test set
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=99)


In [21]:
# Grid Search
# Initialize the classifier
clf = linear_model.LogisticRegression(random_state = 100, class_weight = 'balanced')

# Define Parameter Space
parameters = {'C': [0.01,0.1,1,10,100]}

#Instantiate Grid Search Object
grid_LR = grid_search.GridSearchCV(estimator = clf, param_grid = parameters, n_jobs=-1)


# Fit the object to training data
grid_LR.fit(X_train, y_train)

print('Results using Logistic Regression\n------')
reg_LR = Model(X_train, y_train, X_test, y_test, grid_LR)

# Count Vectorizer
print('Logistic Regression\n-----')
print(reg_LR.best_params_)


Results using Logistic Regression
------
Accuracy score on training data: 0.5034
Accuracy score on testing data: 0.5225

Roc score: 0.5638

Confusion Matrix
[[1035 1013]
 [ 133  219]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.89      0.51      0.64      2048
        1.0       0.18      0.62      0.28       352

avg / total       0.78      0.52      0.59      2400

Logistic Regression
-----
{'C': 0.1}


In [22]:
# Grid Search
# Initialize the classifier
clf = tree.DecisionTreeClassifier(random_state = 100, class_weight = 'balanced')

# Define Parameter Space
# TODO: Create the parameters list you wish to tune
parameters = {'min_samples_leaf': [5, 10, 15, 20, 25, 30],
             "max_leaf_nodes": [2, 5, 15, 20, 25, 30, 35, 50]}
             #"criterion": ['gini', 'entropy']}

#Instantiate Grid Search Object
grid_DT = grid_search.GridSearchCV(estimator = clf, param_grid = parameters, n_jobs=-1)


# Fit the object to training data
grid_DT.fit(X_train, y_train)

print('Results using Decision Tree\n------')
reg_DT = Model(X_train, y_train, X_test, y_test, grid_DT)

# Count Vectorizer
print('Decision Tree\n-----')
print(reg_DT.best_params_)


Results using Decision Tree
------
Accuracy score on training data: 0.6066
Accuracy score on testing data: 0.6175

Roc score: 0.5524

Confusion Matrix
[[1320  728]
 [ 190  162]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.87      0.64      0.74      2048
        1.0       0.18      0.46      0.26       352

avg / total       0.77      0.62      0.67      2400

Decision Tree
-----
{'max_leaf_nodes': 20, 'min_samples_leaf': 5}


In [23]:
# Grid Search
# Initialize the classifier
clf = RandomForestClassifier(random_state = 100, class_weight = 'balanced')

# Define Parameter Space
parameters = {'min_samples_leaf': [10, 20, 25, 50],
             "max_leaf_nodes": [5, 25, 30, 50, 75],
             "n_estimators": [10, 50, 100, 150, 200]}

#Instantiate Grid Search Object
grid_RF = grid_search.GridSearchCV(estimator = clf, param_grid = parameters, n_jobs=-1)


# Fit the object to training data
grid_RF.fit(X_train, y_train)

print('Results using Random Forest\n------')
reg_RF = Model(X_train, y_train, X_test, y_test, grid_RF)

# Count Vectorizer
print('Random Forest\n-----')
print(reg_RF.best_params_)

Results using Random Forest
------
Accuracy score on training data: 0.5178
Accuracy score on testing data: 0.5358

Roc score: 0.5633

Confusion Matrix
[[1074  974]
 [ 140  212]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.88      0.52      0.66      2048
        1.0       0.18      0.60      0.28       352

avg / total       0.78      0.54      0.60      2400

Random Forest
-----
{'max_leaf_nodes': 25, 'min_samples_leaf': 10, 'n_estimators': 100}


In [25]:
reg_RF_final = RandomForestClassifier(n_estimators=100, random_state = 100, class_weight = 'balanced', 
                                      max_leaf_nodes=25, min_samples_leaf=10).fit(X_train, y_train)


#Currant pass
print(reg_RF_final.feature_importances_)

[0.13020207 0.08233846 0.57330322 0.05952379 0.07652105 0.07811142]


In [26]:
df_modeling = df[['creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE', 'creation_source_PERSONAL_PROJECTS',
                 'creation_source_SIGNUP' ,'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'target']]

print(df_modeling.columns)


Index(['creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE',
       'creation_source_PERSONAL_PROJECTS', 'creation_source_SIGNUP',
       'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'target'],
      dtype='object')
